In [1]:
import pandas as pd
#import pandas.io.sql as sqlio
#import sqlalchemy as sql
import psycopg2
#import matplotlib.pyplot as plt
import numpy as np
import os

from dotenv import load_dotenv
#from matplotlib import colors
#from matplotlib.ticker import PercentFormatter
   
from sqlalchemy import create_engine

# load up the variables from the .env file.
load_dotenv()

True

In [2]:
# File to Load (Remember to Change These)
file_to_load = "Resources/StatetoAbbrev.csv"
states_data = pd.read_csv(file_to_load)
states_data.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [3]:
states_data['id']=states_data.index+1
states_data.head()

,State,Abbrev,Code,id
0,Alabama,Ala.,AL,1
1,Alaska,Alaska,AK,2
2,Arizona,Ariz.,AZ,3
3,Arkansas,Ark.,AR,4
4,California,Calif.,CA,5


In [4]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Total Energy Consumed per Capita-StateRankings.csv"

# Read Renewable energy potential file and store into Pandas data frame
Totalenergyconsumed = pd.read_csv(file_to_load)
Totalenergyconsumed.head()

,Rank,State,"Total Energy Consumed per Capita, million Btu",Note: Rankings are based on the full source data values.
0,1,WY,967,NaN
1,2,LA,945,NaN
2,3,ND,872,NaN
3,4,AK,830,NaN
4,5,IA,513,NaN


In [5]:
Totalenergyconsumed = Totalenergyconsumed[['Rank', 'State', 'Total Energy Consumed per Capita, million Btu']]

In [12]:
Totalenergycon_clean = pd.merge(Totalenergyconsumed, states_data,how='inner',left_on='State', right_on='Code')

Totalenergycon_clean = Totalenergycon_clean[['id','Rank','Total Energy Consumed per Capita, million Btu']]

Totalenergycon_clean = Totalenergycon_clean.rename(columns = {'id' : 'State_id'})

Totalenergycon_clean.set_index("State_id", inplace=True)

Totalenergycon_clean.head()

,Rank,"Total Energy Consumed per Capita, million Btu"
State_id,,
51,1,967
19,2,945
35,3,872
2,4,830
16,5,513


In [9]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Total Energy Production-StateRankings.csv"

# Read Renewable energy potential file and store into Pandas data frame
Totalenergyproduction = pd.read_csv(file_to_load)
Totalenergyproduction.head()

,Rank,State,"Total Energy Production, trillion Btu",Note: Rankings are based on the full source data values. Excludes federal offshore production.
0,1,TX,20421,NaN
1,2,PA,8987,NaN
2,3,WY,7718,NaN
3,4,OK,4881,NaN
4,5,WV,4770,NaN


In [10]:
Totalenergyproduction = Totalenergyproduction[['Rank', 'State', 'Total Energy Production, trillion Btu']]

In [13]:
Totalenergyprod_clean = pd.merge(Totalenergyproduction, states_data,how='inner',left_on='State', right_on='Code')

Totalenergyprod_clean = Totalenergyprod_clean[['id','Rank','Total Energy Production, trillion Btu']]

Totalenergyprod_clean = Totalenergyprod_clean.rename(columns = {'id' : 'State_id'})

Totalenergyprod_clean.set_index("State_id", inplace=True)

Totalenergyprod_clean.head()

,Rank,"Total Energy Production, trillion Btu"
State_id,,
44,1,20421
39,2,8987
51,3,7718
37,4,4881
49,5,4770


In [25]:
# File to Load (Remember to Change These)
file_to_load = "Resources/usretechnicalpotential.csv"

# Read Renewable energy potential file and store into Pandas data frame
renewable_energy_data = pd.read_csv(file_to_load)
renewable_energy_data.columns

Index(['Unnamed: 0', 'urbanUtilityScalePV_GWh', 'urbanUtilityScalePV_GW',
       'urbanUtilityScalePV_km2', 'ruralUtilityScalePV_GWh',
       'ruralUtilityScalePV_GW', 'ruralUtilityScalePV_km2', 'rooftopPV_GWh',
       'rooftopPV_GW', 'CSP_GWh', 'CSP_GW', 'CSP_km2', 'onshoreWind_GWh',
       'onshoreWind_GW', 'onshoreWind_km2', 'offshoreWind_GWh',
       'offshoreWind_GW', 'offshoreWind_km2', 'biopowerSolid_GWh',
       'biopowerSolid_GW', 'biopowerSolid_BDT', 'biopowerGaseous_GWh',
       'biopowerGaseous_GW', 'biopowerGaseous_Tonnes-CH4',
       'geothermalHydrothermal_GWh', 'geothermalHydrothermal_GW',
       'EGSGeothermal_GWh', 'EGSGeothermal_GW', 'hydropower_GWh',
       'hydropower_GW', 'hydropower_countOfSites'],
      dtype='object')

### Cleaning data

In [27]:
renewable_energy_data.rename(columns={'Unnamed: 0' : 'State'}, inplace=True)
bio_power_potential = renewable_energy_data[['State', 'biopowerSolid_GWh', 'biopowerGaseous_GWh']]
bio_power_potential.head()

,State,biopowerSolid_GWh,biopowerGaseous_GWh
0,Alabama,11193,1533
1,Alaska,513,61
2,Arizona,1087,837
3,Arkansas,14381,1063
4,California,12408,15510


In [9]:
# set the variables for the pull from SQL.  username and password are in an .env file.  You will need those to make this run
# on your machine.

host = 'localhost'
port = 5432
dbname='ETL_Project'
username = os.environ.get('USER_NAME')
pwd = os.environ.get('PASSWORD')


In [23]:
# set up the connection to the database "ETL_Project"
connection_string = f'{username}:{pwd}@{host}:{port}/{dbname}'
engine = create_engine(f'postgresql://{connection_string}')

In [24]:
engine.table_names()

[]

In [6]:
# set up the connection to the database "ETL_Project"
"""
conn = psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, username, pwd))
sql = "SQL statement"
dat = sqlio.read_sql_query(sql, conn)"""


DatabaseError: Execution failed on sql 'SQL statement': syntax error at or near "SQL"
LINE 1: SQL statement
        ^
